In [1]:
# cnn model
import pandas as pd
import glob
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
#from keras.layers.convolutional import Conv1D
#from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical

ModuleNotFoundError: No module named 'keras'

In [2]:
data_dir = "/Users/krishnagupta/Desktop/updated_training_file/"

In [3]:
df = pd.read_csv(data_dir +"1A1M_test_df.csv", engine="python")
df = pd.read_csv(data_dir + "1A1M_test_df.csv", engine= 'python')

df_x = (df[["cmi.m$value","cc.m$value","cp.m$value","cp1.m$value", "cp2.m$value","hcm.m$value","rsa.m$value",
            "scm.m$value","ssp.m$value", "ecc.m$value", "ecmi.m$value", "ecp.m$value", "ecp1.m$value", "ecp2.m$value", "ehcm.m$value", "ersa.m$value", "escm.m$value", "essp.m$value"]])

df_y = (df[["inf.m$value"]])

df_x.shape, df_y[df_y["inf.m$value"] == 1].shape

((83400, 18), (26, 1))

In [4]:
cls = [str(v) for v in sorted(df['inf.m$value'].unique())]
cls

['0', '1']

In [5]:
file_list = glob.glob(data_dir + "*.csv")

for file_name in file_list:

        df_temp = pd.read_csv(file_name)
        df_xn = (df_temp[["cmi.m$value","cc.m$value","cp.m$value","cp1.m$value", "cp2.m$value","hcm.m$value","rsa.m$value", "scm.m$value","ssp.m$value", "ecc.m$value", "ecmi.m$value", "ecp.m$value", "ecp1.m$value", "ecp2.m$value", "ehcm.m$value", "ersa.m$value", "escm.m$value", "essp.m$value"]])

        df_yn = (df_temp[["inf.m$value"]])
        df_x = pd.concat([df_x,df_xn])
        df_y = pd.concat([df_y,df_yn])
        print("'%-40s' file read with %i True" %(file_name, df_temp[df_temp["inf.m$value"] == 1].shape[0]))


'/Users/krishnagupta/Desktop/updated_training_file/2BVP_test_df.csv' file read with 36 True
'/Users/krishnagupta/Desktop/updated_training_file/2xpx_test_df.csv' file read with 65 True
'/Users/krishnagupta/Desktop/updated_training_file/7uc5_test_df.csv' file read with 28 True
'/Users/krishnagupta/Desktop/updated_training_file/test_df_6JO8.csv' file read with 16 True
'/Users/krishnagupta/Desktop/updated_training_file/7B7N_test_df.csv' file read with 9 True
'/Users/krishnagupta/Desktop/updated_training_file/8cmi_test_df.csv' file read with 24 True
'/Users/krishnagupta/Desktop/updated_training_file/5TEZ_test_df.csv' file read with 24 True
'/Users/krishnagupta/Desktop/updated_training_file/7NTJ_test_df.csv' file read with 3 True
'/Users/krishnagupta/Desktop/updated_training_file/7QS8_test_df.csv' file read with 29 True
'/Users/krishnagupta/Desktop/updated_training_file/7D7S_test_df.csv' file read with 21 True
'/Users/krishnagupta/Desktop/updated_training_file/6BVV_test_df.csv' file read wit

In [6]:
from imblearn.over_sampling import RandomOverSampler
over = RandomOverSampler(sampling_strategy='minority', random_state=42)
x_over, y_over = over.fit_resample(df_x, df_y)

In [7]:
y_over

,inf.m$value
0,0
1,0
2,0
3,0
4,0
...,...
11448177,1
11448178,1
11448179,1
11448180,1


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

#scaler = StandardScaler().fit(x_over)
#rescaledX = scaler.transform(x_over)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(x_over, y_over, train_size=0.80, test_size=0.20, shuffle= True,  random_state=42)

In [10]:
import numpy as np
unique, counts = np.unique(y_test, return_counts=True)
dict(zip(unique, counts))

{0: 1144994, 1: 1144643}

In [11]:
sdscaler = StandardScaler()

X_train_sd = sdscaler.fit_transform(X_train)
X_test_sd = sdscaler.fit_transform(X_test)

In [12]:
X_train_sd.shape[1]

18

In [13]:
import keras
inputs = keras.layers.Input(shape=(X_train_sd.shape[1],1))
RS0    = keras.layers.Reshape((X_train_sd.shape[1], ))(inputs)
FC0    = keras.layers.Dense(16, bias_initializer=keras.initializers.VarianceScaling())(RS0)
BN0    = keras.layers.BatchNormalization(axis=-1)(FC0)
AC0    = keras.layers.Activation('relu')(BN0)
DP0    = keras.layers.Dropout(0.2)(AC0)

RS1    = keras.layers.Reshape((16,1))(DP0)
FC1    = keras.layers.Conv1D(3,3,strides=1)(RS1)
BN1    = keras.layers.BatchNormalization(axis=-1)(FC1)
AC1    = keras.layers.Activation('relu')(BN1)
Pool1 = keras.layers.MaxPool1D(pool_size=1)(AC1)
FL1   = keras.layers.Flatten()(Pool1)

FC3 =   keras.layers.Dense(1, bias_initializer=keras.initializers.VarianceScaling())(FL1)
outputs = keras.layers.Activation('softmax')(FC3)

myCNN1D1 = keras.Model(inputs=inputs,outputs=outputs)
myCNN1D1.compile(optimizer=keras.optimizers.Adam(),
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

myCNN1D1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 18, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 18)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 16, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 14, 3)          │            12 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 14, 3)          │            12 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 14, 3)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 14, 3)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 42)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            43 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 435 (1.70 KB)

 Trainable params: 397 (1.55 KB)

 Non-trainable params: 38 (152.00 B)

In [14]:
import tensorflow as tf
#class_weight = {0: 1, 1: 4}
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
myCNN1D1.fit(X_train_sd, y_train,validation_data=(X_test_sd, y_test),epochs=5,verbose=1,callbacks=[early_stopping_callback])

Epoch 1/5


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


286122/286205 ━━━━━━━━━━━━━━━━━━━━ 0s 322us/step - accuracy: 0.5002 - loss: 0.5823

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


286205/286205 ━━━━━━━━━━━━━━━━━━━━ 106s 368us/step - accuracy: 0.5002 - loss: 0.5823 - val_accuracy: 0.4999 - val_loss: 0.5502
Epoch 2/5
286205/286205 ━━━━━━━━━━━━━━━━━━━━ 108s 376us/step - accuracy: 0.5000 - loss: 0.5637 - val_accuracy: 0.4999 - val_loss: 0.5479
Epoch 3/5
286205/286205 ━━━━━━━━━━━━━━━━━━━━ 107s 372us/step - accuracy: 0.4996 - loss: 0.5622 - val_accuracy: 0.4999 - val_loss: 0.5391
Epoch 4/5
286205/286205 ━━━━━━━━━━━━━━━━━━━━ 118s 413us/step - accuracy: 0.4999 - loss: 0.5616 - val_accuracy: 0.4999 - val_loss: 0.5392
Epoch 5/5
286205/286205 ━━━━━━━━━━━━━━━━━━━━ 113s 394us/step - accuracy: 0.5002 - loss: 0.5610 - val_accuracy: 0.4999 - val_loss: 0.5351
